# Preparing Corpus

## Aims 
* Combine .txt files into one nested list for analysis
* Tokenise, lemmatise and dedupe blog page extracts
* Iterate through possible lda compositions for later anlysis of perplexity and cohesion

## Method

1. Load txt files from crawl into list 
2. Remove stopwords
3. Tokenise list
4. Remove all but nouns from list 
5. Lemmatize terms
6. Recombine into sentances
7. Save prepared dictionaries and sentances for later use
8. Iterate through possible LDA cluster numbers and save each one


## Questions

* Have we prepared the corpus correctly?
* Are we being overly reductive?
* Is this reproducable for other corpus?
* What is the plural of corpus?


The aim of this notebook is to extract, sanitise and prepare the data for processing into LDA and word2vec models. 

In [2]:
from gensim import corpora, models, similarities
import gensim
import logging
import stop_words
import nltk
import string
import os
from nltk.stem import WordNetLemmatizer
import numpy as np
import numpy
import pickle

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

wordnet_lemmatizer = WordNetLemmatizer()

C:\Python27\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Import the texts, concatenate them, remove upper and lower cases, remove stopwords and leave only nouns then finally to lemmatise them to reduce variation in language



In [3]:
txts = os.listdir(".\Data3")

reader = ""

for item in txts:
    with open(".\Data3\{0}".format(item), "rb") as textfile: 
        reader = reader.replace("<![endif][if lt IE 10]><link rel='stylesheet' id='cb-ie-only-css' href='https://www.returnofkings.com/wp-content/themes/15zine/library/css/ie.css?ver=3.2.2' type='text/css' media='all' /><![endif]AboutAdvertiseArchivesDonateContact Us", "")
        reader = reader + textfile.read()

stopwords = stop_words.get_stop_words("en") + u"but of part at to a".split()

print stopwords

cleaned = [x.lower() for x in nltk.word_tokenize(reader)]


# with open("rough_sentances.sent", "wb") as senet: 
#     pickle.dump(cleaned, senet)

cleaned = [x for x in cleaned if x not in stopwords]
print "destopwords"



# cleaned = [x for x in cleaned if x not in string.punctuation]
cleaned = [x for x in cleaned if nltk.pos_tag(x)[0][1] in ["NN"]]
print "only NN"

cleaned = [wordnet_lemmatizer.lemmatize(x) for x in cleaned]
print "lemma-ed"

[u'a', u'about', u'above', u'after', u'again', u'against', u'all', u'am', u'an', u'and', u'any', u'are', u"aren't", u'as', u'at', u'be', u'because', u'been', u'before', u'being', u'below', u'between', u'both', u'but', u'by', u"can't", u'cannot', u'could', u"couldn't", u'did', u"didn't", u'do', u'does', u"doesn't", u'doing', u"don't", u'down', u'during', u'each', u'few', u'for', u'from', u'further', u'had', u"hadn't", u'has', u"hasn't", u'have', u"haven't", u'having', u'he', u"he'd", u"he'll", u"he's", u'her', u'here', u"here's", u'hers', u'herself', u'him', u'himself', u'his', u'how', u"how's", u'i', u"i'd", u"i'll", u"i'm", u"i've", u'if', u'in', u'into', u'is', u"isn't", u'it', u"it's", u'its', u'itself', u"let's", u'me', u'more', u'most', u"mustn't", u'my', u'myself', u'no', u'nor', u'not', u'of', u'off', u'on', u'once', u'only', u'or', u'other', u'ought', u'our', u'ours', u'ourselves', u'out', u'over', u'own', u'same', u"shan't", u'she', u"she'd", u"she'll", u"she's", u'should', u"

The sentances are then split by '.' and then output in a pickle into a sentances object. 

In [4]:
sentances = []

temp = []

for x in cleaned: 
    if "." in x: 
        before = x.split(".")[0]
        after = x.split(".")[1]
        sentances.append(temp)
        temp = []
        temp.append(after)
    else: 
        temp.append(x)
    

# print sentances

with open("sentances.sent", "wb") as senet: 
    pickle.dump(sentances, senet)

The words used within the sentances are then filtered out if they appear in the corpus less than twice. The sentances are then built into gensim corpora objects and saved. 

In [1]:
texts = sentances

# remove words that appear only twice
all_tokens = sum(texts, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 2)

texts = [[word for word in text if word not in tokens_once] for text in texts]

dictionary = corpora.Dictionary(texts)
corp = [dictionary.doc2bow(text) for text in texts]

# pickle.dump(dictionary, "dictionary.dick")
# pickle.dump(corp, "corp.cor")

print len(sentances)


NameError: name 'sentances' is not defined

In [6]:
# gensim.corpora.mmcorpus.MmCorpus.save(corp, "corpp")
dictionary.save("Dicktionary")
with open("corp.cor", "wb") as fp: 
    pickle.dump(corp, fp)

2017-10-07 10:04:13,782 : INFO : saving Dictionary object under Dicktionary, separately None
2017-10-07 10:04:13,849 : INFO : saved Dicktionary


The above data is then used to build LDA models within acceptable ranges for cluster numbers allowing researchers not to be overwhelmed. 

In [5]:
with open("corp.cor") as cp: 
    corp = pickle.load(cp)
with open("sentances.sent") as sent: 
    sentances = pickle.load(sent)
    
dictionary = gensim.corpora.dictionary.Dictionary.load("Dicktionary")

start = 10
stop = 150
jump = 10

for x in range(start, stop, jump): 
    print "processing {}".format(x)
    ldax = gensim.models.ldamodel.LdaModel(corpus=corp,
                                      id2word=dictionary,
                                      num_topics=x,
                                      update_every=1,
                                      chunksize=1000,
                                      passes=5)
    ldax.save("data2lda{}.lda".format(x))

2017-10-08 07:51:20,421 : INFO : loading Dictionary object from Dicktionary
2017-10-08 07:51:20,450 : INFO : loaded Dicktionary
2017-10-08 07:51:20,460 : INFO : using symmetric alpha at 0.1
2017-10-08 07:51:20,463 : INFO : using symmetric eta at 1.64709370316e-05
2017-10-08 07:51:20,474 : INFO : using serial LDA version on this node


processing 10


2017-10-08 07:51:26,187 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 82312 documents, updating model once every 1000 documents, evaluating perplexity every 10000 documents, iterating 50x with a convergence threshold of 0.001000
2017-10-08 07:51:26,190 : INFO : PROGRESS: pass 0, at document #1000/82312
2017-10-08 07:51:28,125 : INFO : merging changes from 1000 documents into a model of 82312 documents
2017-10-08 07:51:28,256 : INFO : topic #5 (0.100): 0.009*"girl" + 0.008*"men" + 0.007*"dont" + 0.007*"king" + 0.007*"must" + 0.007*"visit" + 0.007*"thing" + 0.006*"like" + 0.006*"life" + 0.006*"year"
2017-10-08 07:51:28,259 : INFO : topic #0 (0.100): 0.014*"will" + 0.009*"great" + 0.009*"men" + 0.009*"king" + 0.008*"trump" + 0.007*"like" + 0.007*"make" + 0.007*"medium" + 0.006*"life" + 0.006*"girl"
2017-10-08 07:51:28,260 : INFO : topic #1 (0.100): 0.015*"will" + 0.013*"men" + 0.009*"girl" + 0.007*"time" + 0.007*"visit" + 0.006*"like" +

2017-10-08 07:51:38,619 : INFO : topic #5 (0.100): 0.008*"korea" + 0.006*"girl" + 0.006*"must" + 0.006*"meat" + 0.005*"men" + 0.005*"thing" + 0.005*"guide" + 0.005*"western" + 0.005*"like" + 0.005*"traveling"
2017-10-08 07:51:38,621 : INFO : topic #0 (0.100): 0.019*"men" + 0.019*"visit" + 0.018*"king" + 0.012*"weapon" + 0.012*"great" + 0.012*"girl" + 0.012*"government" + 0.012*"find" + 0.012*"return" + 0.012*"must"
2017-10-08 07:51:38,625 : INFO : topic #9 (0.100): 0.045*"rok" + 0.028*"will" + 0.026*"city" + 0.026*"make" + 0.025*"level" + 0.025*"testosterone" + 0.024*"month" + 0.024*"red" + 0.023*"increase" + 0.023*"provide"
2017-10-08 07:51:38,627 : INFO : topic #6 (0.100): 0.044*"woman" + 0.016*"men" + 0.012*"will" + 0.011*"like" + 0.006*"even" + 0.006*"get" + 0.005*"good" + 0.005*"le" + 0.005*"syria" + 0.005*"year"
2017-10-08 07:51:38,631 : INFO : topic diff=0.208081, rho=0.377964
2017-10-08 07:51:38,634 : INFO : PROGRESS: pass 0, at document #8000/82312
2017-10-08 07:51:39,834 : IN

2017-10-08 07:51:52,042 : INFO : topic #9 (0.100): 0.051*"rok" + 0.030*"testosterone" + 0.029*"will" + 0.028*"red" + 0.027*"city" + 0.027*"increase" + 0.027*"level" + 0.027*"top" + 0.027*"month" + 0.027*"popular"
2017-10-08 07:51:52,048 : INFO : topic diff=0.178757, rho=0.277350
2017-10-08 07:51:52,049 : INFO : PROGRESS: pass 0, at document #14000/82312
2017-10-08 07:51:53,273 : INFO : merging changes from 1000 documents into a model of 82312 documents
2017-10-08 07:51:53,690 : INFO : topic #3 (0.100): 0.008*"even" + 0.007*"europe" + 0.006*"people" + 0.006*"like" + 0.006*"war" + 0.006*"just" + 0.005*"god" + 0.004*"men" + 0.003*"world" + 0.003*"european"
2017-10-08 07:51:53,693 : INFO : topic #2 (0.100): 0.014*"will" + 0.009*"body" + 0.008*"first" + 0.007*"get" + 0.007*"time" + 0.006*"like" + 0.006*"just" + 0.005*"people" + 0.005*"much" + 0.005*"strength"
2017-10-08 07:51:53,694 : INFO : topic #7 (0.100): 0.028*"will" + 0.012*"trump" + 0.008*"life" + 0.006*"year" + 0.006*"hillary" + 0.0

...

2017-10-08 09:12:59,378 : INFO : topic diff=0.203289, rho=0.107211
2017-10-08 09:12:59,384 : INFO : PROGRESS: pass 4, at document #81000/82312
2017-10-08 09:13:01,345 : INFO : merging changes from 1000 documents into a model of 82312 documents
2017-10-08 09:13:01,914 : INFO : topic #13 (0.020): 0.026*"twitter" + 0.025*"method" + 0.024*"leg" + 0.024*"com" + 0.021*"jew" + 0.019*"effective" + 0.016*"hit" + 0.015*"position" + 0.015*"straight" + 0.014*"resource"
2017-10-08 09:13:01,917 : INFO : topic #38 (0.020): 0.021*"security" + 0.020*"resistance" + 0.020*"percent" + 0.019*"stronger" + 0.015*"exact" + 0.014*"provider" + 0.014*"ethnic" + 0.014*"stone" + 0.013*"song" + 0.012*"silence"
2017-10-08 09:13:01,918 : INFO : topic #21 (0.020): 0.067*"medium" + 0.065*"trump" + 0.034*"speech" + 0.034*"hillary" + 0.031*"roosh" + 0.024*"donald" + 0.022*"establishment" + 0.021*"police" + 0.017*"journalist" + 0.016*"president"
2017-10-08 09:13:01,921 : INFO : topic #31 (0.020): 0.048*"will" + 0.037*"get

processing 60


2017-10-08 09:13:41,954 : INFO : running online (multi-pass) LDA training, 60 topics, 5 passes over the supplied corpus of 82312 documents, updating model once every 1000 documents, evaluating perplexity every 10000 documents, iterating 50x with a convergence threshold of 0.001000
2017-10-08 09:13:41,956 : INFO : PROGRESS: pass 0, at document #1000/82312
2017-10-08 09:13:45,176 : INFO : merging changes from 1000 documents into a model of 82312 documents
2017-10-08 09:13:45,747 : INFO : topic #36 (0.017): 0.014*"like" + 0.013*"rok" + 0.011*"king" + 0.009*"woman" + 0.009*"trump" + 0.009*"will" + 0.009*"divorce" + 0.008*"show" + 0.007*"provide" + 0.007*"month"
2017-10-08 09:13:45,750 : INFO : topic #56 (0.017): 0.013*"trump" + 0.013*"will" + 0.011*"just" + 0.009*"people" + 0.009*"political" + 0.008*"thing" + 0.008*"youre" + 0.007*"girl" + 0.007*"make" + 0.007*"like"
2017-10-08 09:13:45,750 : INFO : topic #29 (0.017): 0.015*"woman" + 0.012*"will" + 0.009*"get" + 0.009*"even" + 0.008*"make"

2017-10-08 09:14:14,358 : INFO : topic #25 (0.017): 0.030*"rule" + 0.022*"world" + 0.020*"people" + 0.017*"everyone" + 0.015*"else" + 0.015*"business" + 0.012*"much" + 0.012*"book" + 0.010*"ive" + 0.010*"purchase"
2017-10-08 09:14:14,361 : INFO : topic #36 (0.017): 0.026*"you" + 0.023*"give" + 0.019*"planet" + 0.017*"summer" + 0.016*"pick-up" + 0.016*"speaker" + 0.015*"twitter" + 0.015*"youtube" + 0.015*"education" + 0.013*"buck"
2017-10-08 09:14:14,364 : INFO : topic #54 (0.017): 0.020*"murray" + 0.018*"song" + 0.016*"5" + 0.015*"week" + 0.015*"men" + 0.014*"just" + 0.013*"time" + 0.013*"meeting" + 0.011*"granted" + 0.010*"like"
2017-10-08 09:14:14,367 : INFO : topic #5 (0.017): 0.117*"woman" + 0.030*"men" + 0.019*"beautiful" + 0.016*"islam" + 0.014*"life" + 0.012*"benefit" + 0.011*"like" + 0.008*"le" + 0.008*"religion" + 0.008*"likely"
2017-10-08 09:14:14,368 : INFO : topic #43 (0.017): 0.057*"rok" + 0.032*"will" + 0.031*"testosterone" + 0.031*"city" + 0.031*"make" + 0.030*"level" + 

KeyboardInterrupt: 